<a href="https://colab.research.google.com/github/stephenfrein/csc8491/blob/main/AssociationRulesSourceCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install special packages - comment these lines out after first install
install.packages("tidyr")

# load packages for use
library(tidyr)


In [ ]:
# get the file from the web
URL <- "https://csc8491.s3.us-east-1.amazonaws.com/Tomcat.csv"
rawdata <- read.csv(URL, stringsAsFactors=FALSE)

In [ ]:
# visually inspect the raw data
head(rawdata)

In [ ]:
# strip the data down to a few useful columns
readydata <- rawdata[,c(2,3,10)]

# visually inspect the changed data
head(readydata)

In [ ]:
# the files column has many files in a single column
# turn it into multiple columns
# don't worry about warning message about "missing pieces"
# we are making space for up to 100 files per bug
# most have far fewer files than that
readydata <- readydata %>% separate(files, paste('V', 1:100), sep = " ")
head(readydata)

In [ ]:
# now drop columns that aren't files
readydata <- readydata[,-c(1:2)]

# visually inspect the changed data
head(readydata)

In [ ]:
# convert the rest into a format that can be read by arules package
newtemp <- tempfile(fileext = ".csv")
write.csv(readydata,file = newtemp)

In [ ]:
# install association rules package
# comment below line out after first install
install.packages("arules")
library(arules)

In [ ]:
# read transactions - skip the first line of header info
checkins <- read.transactions(newtemp, sep = ",", skip = 1)
summary(checkins)

# look at the first five check-ins
inspect(checkins[1:5])

In [ ]:
# plot the frequency of items
itemFrequencyPlot(checkins, topN = 20)

In [ ]:
# training a model on the data
# default settings result in zero rules learned
apriori(checkins)

In [ ]:
# set support and confidence levels
filerules <- apriori(checkins, parameter = list(
  support = 0.005, confidence = 0.50))

# look at the first three rules
inspect(filerules[1:3])

In [ ]:
# sorting rules by lift
inspect(sort(filerules, by = "lift")[1:5])

In [ ]:
# finding subsets of rules containing specific items
some_file_rules <- subset(filerules, items %in%
                    "java/org/apache/coyote/http11/Http11Processor.java")
inspect(some_file_rules)